# Assignment 2.3 - Vehicle Routing Problem 
JM0100-M-6 Business Analytics  
Myrthe Wouters  
u1273195

In [1]:
# Global imports
import pandas as pd
import numpy as np
from haversine import haversine, Unit
import math
import operator
from copy import deepcopy
import random
from deap import base, creator, tools, algorithms

In [2]:
# Load data
STORES = pd.read_excel('Data Excercise 2 - EMTE stores - BA 2020-1.xlsx', index_col='City Nr.')

In [3]:
STORES['is_jumbo'] = STORES.apply(lambda row: row['Type']=='Jumbo', axis=1) 

In [4]:
STORES.head()

,Name,Address,Postal code,City,Lat,Long,Type,is_jumbo
City Nr.,,,,,,,,
0,EMTE HEADQUARTERS VEGHEL,CORRIDOR 11,5466RB,VEGHEL,51.606702,5.528046,NaN,False
1,EMTE ARKEL,DR H DE VRIESPLN 14,4241BW,ARKEL,51.864000,4.993040,Coop,False
2,EMTE ARNEMUIDEN FR,CLASINASTR 5,4341ER,ARNEMUIDEN,51.500010,3.677280,Jumbo,True
3,EMTE BATHMEN FR,LARENSEWG 18,7437BM,BATHMEN,52.249060,6.289990,Jumbo,True
4,EMTE BEEK EN DONK,HEUVELPLN 73,5741JJ,BEEK EN DONK,51.529300,5.632300,Jumbo,True


## Exercise 2.3
Improve your heuristic algorithm (the result from Exercise 2.2) by applying a Genetic Algorithm approach. 

### Calculate rounded distances between every two locations
I will use a matrix to store the distances between every two stores. This makes my algorithms more efficient, because I do not have to call the haversine formula every time I want to compute the distance between two stores.

In [5]:
def calc_dist(loc1, loc2):
    '''Defines rounded distance (km) between two locations'''
    coords1 = STORES.loc[loc1, 'Lat'], STORES.loc[loc1, 'Long']
    coords2 = STORES.loc[loc2, 'Lat'], STORES.loc[loc2, 'Long']
    dist = round(haversine(coords1, coords2))
    return dist

In [6]:
def dist_to_min(km, speed_kmh):
    """Defines the duration of a route in minutes"""
    speed_kmm = speed_kmh/60
    minutes = round(km/speed_kmm)
    return minutes

In [7]:
# Save all distance in global variable DIST_MATRIX and all travel_times in global variable TIME_MATRIX
DIST_MATRIX = np.zeros((len(STORES), len(STORES)))
TIME_MATRIX = np.zeros((len(STORES), len(STORES)))

for location_1 in STORES.index:
    for location_2 in STORES.index:
        dist = calc_dist(location_1, location_2)
        DIST_MATRIX[location_1, location_2] = dist
        TIME_MATRIX[location_1, location_2] = dist_to_min(dist, 90)

### Read results of 2.1 and 2.2 to Excel file
In order to run the GA of this exercise 2.3, I have to read the solutions of both exercise 2.1 and 2.2 to a nested list of lists with store numbers. I exclude the headquarters from these lists for DEAP/GA purposes.

In [8]:
def read_from_excel(file):
    '''
    Function that reads the solution of earlier excercises in an Excel file and returns a nested list of lists with store numbers, excluding HQ
    '''
    # Read from excel file 
    solution = pd.read_excel(file)
    solution = list(solution['City Nr.'])[1:-1] # Take store numbers, exact for the first and last HQ

    schedule = [] 
    route = []
    for location in solution:
        # Append all store numbers except for headquarters
        if location!=0:
            route.append(location)
        # Once we come accross a headquarter
        else:
            # If the HQ is the end of the route, append this route to initial_schedule
            if len(route)>0:
                schedule.append(route[:])
                route = []
    # Append last route to schedule
    schedule.append(route[:])
    
    return schedule

In [9]:
solution_1 = read_from_excel('Ex2.1-1273195.xls')
solution_2 = read_from_excel('Ex2.2-1273195.xls')

For implementation puproses, I need to check for each route if it contains Jumbo or Coop/Other type stores. Therefore, I give each route a 'route type'. I assumed that the route types for solution 1 are the same as the route types for solution 2. I also checked this in the code block below and we can see that the route types are the same in both exercises. I store the route types in the global variable `ROUTE_TYPES`.

In [10]:
route_types_1 = {nr: 'Jumbo' if STORES.loc[solution_1[nr][0], 'is_jumbo'] else 'Coop/Other' for nr in range(11)}
route_types_2 = {nr: 'Jumbo' if STORES.loc[solution_2[nr][0], 'is_jumbo'] else 'Coop/Other' for nr in range(11)}
print(route_types_1 == route_types_2)
ROUTE_TYPES = route_types_1

True


### Helper functions
In order to define proper custom mutation, crossover and evaluation functions for the implemented GA, I need some helper functions that are defined below. 

In [11]:
HQ = 0
MAX_WORKING_MINS = 11*60
MAX_OPENING_MINS = 8*60
MAX_OPENING_MINS = 8*60

def flatten_schedule(schedule_nested):
    '''Flattens a schedule of nested routes with location numbers'''
    return [loc for route in schedule_nested for loc in route]

def nest_schedule(flat_schedule, lengths):
    '''Nest a flatttened schedule back to a schedule of nested routes with location numbers'''
    nested_schedule = []
    start = 0
    for length in lengths:
        nested_schedule.append(flat_schedule[start:(start+length)])
        start+=length
    return nested_schedule
        

def total_distance(route, full_route=False):
    '''
    Calculates total distance of a schedule.
    Takes an argument full_route that can be set to True or False. If set to True, calculates the total distance of the route including the 
    start and end at headquarters. If set to False, calculates the total distance of the route excluding start and end at headquarters. 
    '''
    route_to_check = deepcopy(route)
    
    if full_route:
        route_to_check.insert(0, HQ)
        route_to_check.append(HQ)
    distances = [DIST_MATRIX[route_to_check[idx], route_to_check[idx+1]] 
                 for idx, _ in enumerate(route_to_check[:-1])]
    
    return sum(distances)
    

def travel_time(route, full_route=False):
    '''
    Defines the total travel time of the schedule, i.e., the time spend travelling in driving to stores in route.
    Takes an argument full_route that can be set to True or False. If set to True, calculates the travel time of the route including the 
    start and end at headquarters. If set to False, calculates the travel time of the route excluding start and end at headquarters. 
    '''
    route_to_check = deepcopy(route)
    
    if full_route:
        route_to_check.insert(0, HQ)
        route_to_check.append(HQ)

    travel_times = [TIME_MATRIX[route_to_check[idx], route_to_check[idx+1]]
                    for idx, _ in enumerate(route_to_check[:-1])]
    
    return sum(travel_times)

def visiting_time(route):
    '''
    Defines the visiting time of all stores in a route
    '''
    is_jumbo = STORES.loc[route[0], 'is_jumbo']
    time = 30 if is_jumbo else 20
    return len(route)*time

def working_hours_constraint(route):
    '''Defines if route meets the constraint of John's working hours'''
    return (travel_time(route, full_route=True) + visiting_time(route)) <= MAX_WORKING_MINS

def opening_hours_constraint(route):
    '''Defines if route meets the constraint of visiting hours 09:00-17:00 at every store in route'''
    return (travel_time(route) + visiting_time(route)) <= MAX_OPENING_MINS

def valid_hours(route):
    '''
    Checks if the route meets the two hours constraints:
    * John's working hours 
    * Visiting hours 09:00-17:00 at every store
    '''
    return working_hours_constraint(route) and opening_hours_constraint(route)

def store_type_constraint(route):
    '''Defines if route meets the store type constraint, i.e., all stores in the route should be of the same type (Jumbo or Coop/Other)'''
    is_jumbo = STORES.loc[route[0], 'is_jumbo']
    
    for location in route[1:]:
        if STORES.loc[location, 'is_jumbo'] != is_jumbo:
            return False
    
    return True

def is_valid(route):
    '''
    Checks if the route meets all three constraints:
    * John's working hours
    * Visiting hours 09:00-17:00 at every store
    * All stores in the route being of the same type
    '''
    return working_hours_constraint(route) and opening_hours_constraint(route) and store_type_constraint(route)

### Genetic Algorithm

I implemented my Genetic Algorithm using the DEAP library (https://deap.readthedocs.io/en/master/index.html). First, I experimented with the pre-built operators for mutation and cross-over defined by DEAP. I represented my individuals as a flat list of 133 store numbers (so excluding HQs) and worked with an evaluation function that would make sure that each individual schedule meets the constraints (i.e., including as many routes as needed to meet constraints). However, this approach lead to far from satisfying results as many far from optimal schedules are included in the population and is therefore not included in this notebook.

Then, I realised that the algorithm would probably do better by implementing custom functions for mutation and cross-over that would handle the constraints for this specific problem in order not to waste search time on infeasible solutions. Therefore, in my final GA shown below, I represent an individual schedule as a  nested list, with each sub-list representing a route. Each individual in the population consist of a schedule with 11 routes and meets all constraints. 

In the following sections I will define the custom functions for crossover and mutation. Thereafter, I will instantiate the toolbox and execute the GA.

#### Custom operators

##### Mutation
I defined a custom mutation function `mutate` that considers multiple swaps and moves within an individual. Therefore, I first define `swap_with_random` and `move_with_random` functions.

In [12]:
def swap_locations(schedule, idx_route1, idx_route2, idx_elem1, idx_elem2):
    '''Swaps two locations in given routes'''
    schedule[idx_route1][idx_elem1], schedule[idx_route2][idx_elem2] = schedule[idx_route2][idx_elem2], schedule[idx_route1][idx_elem1]
    
def swap_with_random(schedule, idx_route, idx_elem, route_type):
    '''Swaps a given location in a given route with a random other location'''
    
    # Define possible routes to swap with, i.e., routes with the same type as the route the given location is in
    possible_routes_to_swap = [route for route, typ in ROUTE_TYPES.items() if typ==route_type]
    idx_route_to_swap = random.choice(possible_routes_to_swap) # Pick a random route from all possible routes
    
    # Pick a random location in the chosen route to swap with
    # Exclude the possibility of the given individiual swapping with itself
    if idx_route == idx_route_to_swap:
        possible_elems_to_swap = list(range(len(schedule[idx_route_to_swap])))
        possible_elems_to_swap.remove(idx_elem)
        idx_elem_to_swap = random.choice(possible_elems_to_swap)
    else:
        idx_elem_to_swap = random.choice(range(len(schedule[idx_route_to_swap])))
    
    swap_locations(schedule, idx_route, idx_route_to_swap, idx_elem, idx_elem_to_swap) # Swap locations
    
    # Check if the changed routes are still valid
    # We do not have to check for store type as we only considered possible routes to swap with, with same route type
    changed_routes_valid = all((valid_hours(schedule[idx_route]), valid_hours(schedule[idx_route_to_swap])))
    if changed_routes_valid:
        dist = sum([total_distance(route, full_route=True) for route in schedule])
    else: 
        dist=None
    # Store details of the swap
    swap_details = idx_route, idx_route_to_swap, idx_elem, idx_elem_to_swap
    
    # Revert the swap
    swap_locations(schedule, idx_route_to_swap, idx_route, idx_elem_to_swap, idx_elem)
    
    return changed_routes_valid, dist, swap_details, 'swap'

def move_locations(schedule, orig_idx_route, idx_route_to_move, elem, new_pos):
    '''Moves location to a position in idx_route_to_move'''
    schedule[orig_idx_route].remove(elem)
    schedule[idx_route_to_move].insert(new_pos, elem)
    
def move_to_random(schedule, idx_route, idx_elem, elem, route_type):
    '''Swaps a given location in a given route with to a random other position in the same or another route'''
    
    # Define possible routes to swap with, i.e., routes with the same type as the route the given location is in
    possible_routes_to_move = [route for route, typ in ROUTE_TYPES.items() if typ==route_type]
    idx_route_to_move = random.choice(possible_routes_to_move) # Pick a random route from all possible routes
    
    # Pick a random position in the chosen route to move to
    # Exclude the possibility of the given individiual moving to its own location
    if idx_route == idx_route_to_move:
        possible_positions = list(range(len(schedule[idx_route_to_move])+1))
        possible_positions.remove(idx_elem)
        new_pos = random.choice(possible_positions)
    else:
        new_pos = random.choice(range(len(schedule[idx_route_to_move])+1))
    
    move_locations(schedule, idx_route, idx_route_to_move, elem, new_pos) # Move location
    
    # Check if the changed routes are still valid
    # We do not have to check for store type as we only considered possible routes to move to with same route type
    changed_route_valid = valid_hours(schedule[idx_route_to_move])
    if changed_route_valid:
        dist = sum([total_distance(route, full_route=True) for route in schedule])
    else: 
        dist=None
        
    # Store details of the swap
    move_details = idx_route, idx_route_to_move, elem, new_pos
    
    # Revert the move
    move_locations(schedule, idx_route_to_move, idx_route, elem, idx_elem)
    
    return changed_route_valid, dist, move_details, 'move'

def mutate(individual, indpb):
    '''
    Function that mutates an individual and returns a valid individual, i.e., an individual that meets all constraints. 
    Takes an argument indpb representing the probability of each location to be either swapped or moved.
    If a location is picked to mutate, 100 swaps or moves are considered (both with 50% probability per iteration). From those 100 moves/swaps,
    the one that leads to the best total distance is applied. 
    '''
    
    # Check for all locations
    for idx_route, route in enumerate(individual):
        route_type = ROUTE_TYPES[idx_route]
        for idx_elem, loc in enumerate(route):
            # Define if location is considered for mutation
            if random.random() < indpb:
                # Initialise empty informations list that will store informatior from all 100 swaps/moves
                informations = []
                
                # Do 100 swaps/moves
                for i in range(100):
                    # Decide if this iteration will consider a swap or a move (50% chance for both)
                    swap = random.randint(0,1)
                    if swap:
                        # Swap with random individual
                        information = swap_with_random(individual, idx_route, idx_elem, route_type)
                        informations.append(information) # Append information of swap to informations list
                    else: 
                        # Move to random position
                        information = move_to_random(individual, idx_route, idx_elem, loc, route_type)
                        informations.append(information) # Append information of move to informations list
                        
                # Consider only valid swaps and moves
                valid = [info[1:] for info in informations if info[0]]
                
                # If there is a valid swap or move, execute the swap or move that leads to the best total distance
                if len(valid):
                    best = sorted(valid, key=lambda x: x[0])[0]
                    if best[-1] == 'move':
                        move_locations(individual, *best[1])
                    else:
                        swap_locations(individual, *best[1])
                        
    return individual,

##### Crossover
The crossover function of DEAP only takes individuals as flat lists as inputs and returns offspring that might not meet all constraints for this specific problem. Therefore, I implemented a custom crossover function that first flattens the two input individuals. Then takes at most 500 tries to mate the two individuals with the cxPartialyMatched operator defined by DEAP. Then the offspring are nested again. As soon as this procedure leads to two valid offsprings, return. 

In [13]:
def crossover(ind1, ind2):
    '''
    Takes as input two individuals and returns two valid offspring. If no two valid offsprings can be found within 500 iterations, returns the
    individuals themselves. 
    Uses the cxPartialyMatched crossover operator defined by DEAP.
    '''
    
    # DEAP requires flat list
    # Save lengths of the routes within individuals 1 and 2 in order to nest the offspring back later
    lengths_ind1 = [len(route) for route in ind1]
    lengths_ind2 = [len(route) for route in ind2]
    
    flat_ind1, flat_ind2 = flatten_schedule(ind1), flatten_schedule(ind2) # flatten individuals
    # DEAP requires store numbers starting from 0, therefore subtract 1 from each store number
    flat_ind1_deap = [nr-1 for nr in flat_ind1]
    flat_ind2_deap = [nr-1 for nr in flat_ind2]
    
    # Try 500 crossovers, as soon as we find two valid offsprings return
    for i in range(500):
        # cxPartialyMatched on the two flattened individuals
        flat_child1_deap, flat_child2_deap = tools.cxPartialyMatched(flat_ind1_deap[:], flat_ind2_deap[:])
        
        # Get back original store numbers by adding 1 from each store number
        flat_child1 = [nr+1 for nr in flat_child1_deap]
        flat_child2 = [nr+1 for nr in flat_child2_deap]
        
        # Nest children
        child1 = nest_schedule(flat_child1, lengths_ind1)
        child2 = nest_schedule(flat_child2, lengths_ind2)
        
        # If both offsprings are valid, return the offspring
        if all(is_valid(route) for route in child1) and all(is_valid(route) for route in child2):
            return creator.Individual(child1), creator.Individual(child2)
        
    # If no two valid offsprings are found within 500 iterations, return individuals
    return ind1, ind2

##### Evaluation
As an evaluation metric for the Genetic Algorithm, I use total distance of the individual.

In [14]:
def evaluate_distance(individual):
    '''Evaluates total distance of an individual'''
    return sum([total_distance(route, full_route=True) for route in individual]), 

#### Genetic Algorithm implementation

##### Creator
As we are considering a distance minimization problem, we define the class `FitnessMin` and set weights to `(-1.0,)`.

In [86]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

##### Initialise population

I initialise the population with 20% individuals from the solution of 2.1 and 80% random mutations with indpb of 0.05 of the solution of 2.1. 

I also tried other initialisation strategies that are not shown in this notebook. For example, random mutations with indpb of 0.05 of the solutions of 2.1 and 2.2 (approximately 50% 2.1 and 50% 2.2) and random mutations of the solution of 2.2 only. For these two strategies, I did not manage to improve the result of 2.2. Hence, it seems that in this case the population does not have enough variance over the generations and therefore we end up in a local minimum.

In [17]:
def init_individual(solution_1, indpb):
    '''Initialises an individual as a random mutation with indpb of either solution_1 or solution_2'''
    ind = deepcopy(solution_1)
    if random.random() < 0.8:
        mutate(ind, indpb)
    return ind  
    
# Instantiate toolbox instance
toolbox = base.Toolbox()

toolbox.register('init_individual', init_individual, solution_1, 0.05)
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.init_individual)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

##### Genetic operators
For selection, I implement Tournament Selection with a tournament size of 2. This gives a bigger chance to individuals with low fitness values in the population. I also experimented with a custom ranking-based selection algorithm, however this seemed to work worse than tournament selection.

In [18]:
toolbox.register("evaluate", evaluate_distance)
toolbox.register("mate", crossover)
toolbox.register("mutate", mutate, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=2)

##### Genetic Algorithm
Below, I define the function `main` that executes the entire Genetic Algorithm. Also, I implement the GA using the `eaSimpleWithElitism` algorithm. This algorithm is a variation of the `eaSimple` algorithm defined by DEAP, but uses Elitism to 'protect' the best individual in the population. Code for this function is taken from the GitHub repository Hands-On-Genetic-Algorithms-with-Python and can be found at: https://github.com/PacktPublishing/Hands-On-Genetic-Algorithms-with-Python/blob/master/Chapter05/elitism.py

I also implement a HallOfFame instance that stores the individual with the lowest fitness value over all generations. After a lot of experimentation, I work with a population size of $500$, a `cxpb` of $0.7$ and a `mutpb` of $0.3$. 

With these settings and an initial population consisting of the solution of 2.1 only, I managed to improve the solution of 2.2 (distance of $2838$) to a distance of $2832$.

In [15]:
def eaSimpleWithElitism(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
    """This algorithm is similar to DEAP eaSimple() algorithm, with the modification that
    halloffame is used to implement an elitism mechanism. The individuals contained in the
    halloffame are directly injected into the next generation and are not subject to the
    genetic operators of selection, crossover and mutation.
    """
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is None:
        raise ValueError("halloffame parameter must not be empty!")

    halloffame.update(population)
    hof_size = len(halloffame.items) if halloffame.items else 0

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    # Begin the generational process
    for gen in range(1, ngen + 1):

        # Select the next generation individuals
        offspring = toolbox.select(population, len(population) - hof_size)

        # Vary the pool of individuals
        offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # add the best back to population:
        offspring.extend(halloffame.items)

        # Update the hall of fame with the generated individuals
        halloffame.update(offspring)

        # Replace the current population by the offspring
        population[:] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)

    return population, logbook

In [19]:
def main():
    pop = toolbox.population(n=500)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register('avg', np.mean)
    stats.register('std', np.std)
    stats.register('min', np.min)
    stats.register('max', np.max)
    
    pop, log = eaSimpleWithElitism(pop, toolbox, cxpb=0.7, mutpb=0.3, ngen=1000, 
                                   stats=stats, halloffame=hof, verbose=True)
    
    return pop, log, hof

In [20]:
pop, log, hof = main()

gen	nevals	avg    	std   	min 	max 
0  	500   	3057.85	42.838	3000	3264
1  	399   	3053.77	37.6607	2991	3272
2  	380   	3051.64	40.7469	2991	3226
3  	389   	3049.39	42.0453	2989	3281
4  	405   	3043.72	37.8153	2984	3276
5  	402   	3041.84	38.5565	2984	3303
6  	364   	3038.36	40.3623	2981	3234
7  	386   	3036.68	41.7183	2979	3303
8  	405   	3038.19	41.5647	2979	3267
9  	391   	3038.04	44.8433	2970	3320
10 	383   	3027.21	33.909 	2966	3164
11 	395   	3024.76	33.9683	2966	3184
12 	387   	3023.5 	38.3492	2958	3266
13 	394   	3021.06	37.1833	2958	3168
14 	389   	3018.49	38.9383	2958	3252
15 	369   	3017.29	41.164 	2958	3368
16 	393   	3015.62	38.108 	2957	3234
17 	389   	3014.63	42.7964	2950	3328
18 	408   	3009.26	36.7634	2950	3215
19 	393   	3007.42	38.0414	2944	3180
20 	406   	3008   	40.7004	2944	3307
21 	399   	3006.75	38.9899	2944	3203
22 	379   	3004.23	41.6853	2944	3198
23 	386   	3000.19	36.8013	2944	3171
24 	403   	3000.56	44.4058	2932	3292
25 	406   	3000.53	41.9556	2925	3210
26 

In [40]:
schedule_ga = [route for nested in hof.items for route in nested]
print('The distance of the schedule after implementing a Genetic Algorithm approach equals {} km.'.format(
    sum([total_distance(route, full_route=True) for route in schedule_ga])))
print('This is an improvment of {} km compared to the result in exercise 2.2.'.format(
    sum([total_distance(route, full_route=True) for route in solution_2]) -
    sum([total_distance(route, full_route=True) for route in schedule_ga])))

The distance of the schedule after implementing a Genetic Algorithm approach equals 2832.0 km.
This is an improvment of 6.0 km compared to the result in exercise 2.2.


Although this GA finds an improvement of 6 km compared to the result in exercise 2.2, it is not the best schedule in terms of distance that I found. Next to implementing a full GA as described above, I have tried to implement a GA for each route in the schedule. I will elaborate upon this approach below.  

### Genetic Algorithm for each route

As mentioned, I have implemented a Genetic Algorithm for each route in the solution of 2.2 (i.e., in total 11 routes) separately. This is in fact the same as considering a Traveling Salesman Problem for each route. It turned out that only for route 1 this strategy leads to an improvement. Therefore, the GAs for the other routes are not included in this notebook. Resuls of the Genetic Algorithm for route 1 only are shown below.

In [143]:
route_1 = deepcopy(solution_2)[0]
print('The distance for route 1 in solution 2.2 is {} km'.format(total_distance(route_1, full_route=True)))

The distance for route 1 in solution 2.2 is 458.0 km


Because the DEAP algorithm works with numbers starting from 0 only, I create a deap representation for route 1 together with a dictionary that can  convert the DEAP numbers back to the actual store numbers. Furthermore, I define a function that converts the route in DEAP numbers to the route in actual store numbers.

In [127]:
route_1_deap = [idx for idx, nr in enumerate(route_1)] # Route representation for deap
nr_deap = {idx: nr for idx, nr in enumerate(route_1)} # Dictionary to go from nr in deap to actual location number

In [139]:
def ind_to_route(ind):
    '''Converts the individual with deap numbers to the route with actual store numbers'''
    return [nr_deap[nr] for nr in ind]

For the rest, the genetic algorithm for route 1 uses the same helper functions as defined in the beginning of this notebook.

#### Operators

##### Crossover and Mutation
For the Genetic Algorithm for route 1, I did not implement any custom crossover and mutation operators. For crossover, I simply consider the DEAP operator cxPartialyMatched. For mutation, I consider the DEAP operator mutShuffleIndexes.

##### Evaluation
For this GA, the evaluation function returns the distance of the route. It also handles constraints by 'cutting' a route into multiple routes whenever the constraints are violated.

In [129]:
def evaluate_route_distance(individual):
    '''
    Evaluates distance of a route.
    Handles constraints in the sense that it 'cuts' a route into multiple routes whenever the constraints are violated.
    '''
    sub_route = ind_to_route(individual)
    
    schedule = []
    route = []
    
    for loc in sub_route:
        route.append(loc)
        if not is_valid(route):
            route.remove(loc)
            schedule.append(deepcopy(route))
            route = [loc]
    schedule.append(deepcopy(route))

    distances = [total_distance(route, full_route=True) for route in schedule]
    
    return sum(distances),

#### Genetic Algorithm implementation

##### Creator
As we are considering a distance minimization problem, we define the class `FitnessMin` and set weights to `(-1.0,)`.

In [130]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

##### Initialisation
I initialise the population with random permuations of route_1. That is, I randomly shuffle route_1 to create an individual.

In [131]:
def init_individual_route(route_1_deap):
    '''Initialises an individual by randomly shuffling the store numbers in the route'''
    ind = deepcopy(route_1_deap)
    random.shuffle(ind)
    return ind
    
# Instantiate toolbox instance
toolbox = base.Toolbox()

toolbox.register('init_individual', init_individual_route, route_1_deap)
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.init_individual)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

##### Genetic operators
As mentioned before, I use For selection, I implement crossover and mutation with DEAP operators cxPartialyMatched and mutShuffleIndexes (indpb of $0.05$) respectively. Furthermore, I use Tournament Selection with a tournament size of 3. 

In [132]:
toolbox.register("evaluate", evaluate_route_distance)
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

##### Genetic Algorithm
Below, I define the function `main_route` that executes the entire Genetic Algorithm for route 1. I implement the GA using the `eaSimple` algorithm defined by DEAP. Next to this, I implement a HallOfFame instance that stores the individual with the lowest fitness value over all generations. After a lot of experimentation, I work with a population size of $300$, a `cxpb` of $0.2$ and a `mutpb` of $0.7$. 

In [133]:
def main_route():
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.2, mutpb=0.7, ngen=100, 
                                   stats=stats, halloffame=hof, verbose=True)
    
    return pop, log, hof

In [134]:
pop_1, log_1, hof_1 = main_route()

gen	nevals	avg   	std    	min	max 
0  	300   	928.39	55.5842	760	1049
1  	238   	894.833	57.5139	748	1042
2  	222   	856.183	56.3975	747	1023
3  	218   	833.193	61.2486	542	1033
4  	229   	815.277	64.5427	542	993 
5  	240   	802.363	68.2171	542	1007
6  	229   	789.893	67.923 	542	1047
7  	223   	782.627	72.8749	542	994 
8  	238   	760.71 	74.2658	528	945 
9  	223   	751.94 	89.2314	515	960 
10 	221   	731.863	102.046	509	971 
11 	231   	720.353	123.237	498	1007
12 	231   	685.927	127.344	489	976 
13 	224   	662.37 	133.586	479	933 
14 	241   	647.763	141.276	479	982 
15 	232   	642.02 	147.201	479	961 
16 	221   	609.277	135.432	479	946 
17 	232   	613.333	138.284	469	946 
18 	236   	605.767	137.03 	469	957 
19 	230   	600.8  	139.24 	469	917 
20 	233   	594.223	144.648	469	970 
21 	231   	583    	140.297	469	968 
22 	227   	565.737	135.648	469	984 
23 	230   	566.94 	131.96 	465	891 
24 	224   	570.237	137.554	464	918 
25 	242   	564.857	132.123	455	946 
26 	226   	564.303	138.694	455

In [144]:
best_route_1 = ind_to_route([route for nested in hof_1.items for route in nested])
print('The distance of route 1 improved to {} km'.format(total_distance(best_route_1, full_route=True)))
print('This is an improvement of {} km'.format(total_distance(route_1, full_route=True)-total_distance(best_route_1, full_route=True)))

The distance of route 1 improved to 449.0 km
This is an improvement of 9.0 km


### Final Best Solution
The final schedule with a best total distance is the schedule from route 1, where route 1 is changed to best route 1 from the GA of route 1.

In [149]:
best_schedule = deepcopy(solution_2)
best_schedule[0] = best_route_1
print('The best schedule that I achieved has a distance of {} km.'.format(
    sum([total_distance(route, full_route=True) for route in best_schedule])))
print('This is an improvement of {} km compared to the solution of exercise 2.2.'.format(
    sum([total_distance(route, full_route=True) for route in solution_2]) -
    sum([total_distance(route, full_route=True) for route in best_schedule])))

The best schedule that I achieved has a distance of 2829.0 km.
This is an improvement of 9.0 km compared to the solution of exercise 2.2.


#### Write to Excel

In [177]:
def to_Excel(planned_routes, save=True, file_name=None):
    '''Creates dataframe with needed information from solved VRP, saves to Excel file if needed'''
    
    # Dictionary to store data from all routes
    data = {'Route Nr.': [], 
            'City Nr.': [],
            'City Name': [], 
            'Total Distance in Route (km)': [],
            'Total Distance (km)': []}
    
    # Define total kilometers traveled at start of the route in previous routes
    total_km_at_start = 0
    route_nr = 1
    
    for route in planned_routes:
        # Add start and end at headquarters to route
        full_route = deepcopy(route)
        full_route.insert(0, 0)
        full_route.append(0)
        
        # Define distances and cumulative sum of distances
        distances = [DIST_MATRIX[full_route[idx], full_route[idx+1]] 
                     for idx, _ in enumerate(full_route[:-1])]
        cumsum = list(np.cumsum(distances))
        cumsum.insert(0,0) # Insert cumulative distance of 0 at start at headquarters

        cumsum_loc = 0
        for location in full_route:
            data['Route Nr.'].append(route_nr)
            data['City Nr.'].append(location)
            data['City Name'].append(STORES.loc[location, 'Name'])
            data['Total Distance in Route (km)'].append(int(cumsum[cumsum_loc]))
            data['Total Distance (km)'].append(int(cumsum[cumsum_loc] + total_km_at_start))
            
            cumsum_loc += 1 # Update current cumulative sum location
            
        # Update route numbers
        route_nr += 1
        # Update total kilometers traveled at start of the route in previous routes
        total_km_at_start += cumsum[-1] 

    df = pd.DataFrame.from_dict(data) # Save data to DataFrame
    
    # If needed, save data to Excel file
    if save:
        df.to_excel(file_name, index=False)
            
    return df

In [178]:
df = to_Excel(best_schedule, file_name='Ex2.3-1273195.xls')